In [ ]:
import scipp as sc
import scippneutron as scn
from ess.diffraction import powder
from ess import powgen
import ess
import numpy as np

In [ ]:
sample = scn.load(ess.powgen.data.sample_file(),
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={'LoadMonitors': True})

In [ ]:
cal = powder.load_calibration(
    ess.powgen.data.calibration_file(),
    mantid_args={'InstrumentFilename': 'POWGEN_Definition_2011-02-25.xml'})

In [ ]:
cal = powgen.beamline.map_detector_to_spectrum(cal, detector_info=sample.coords['detector_info'].value)

In [ ]:
sample

Normalize by proton charge

In [ ]:
sample /= sample.attrs['gd_prtn_chrg'].values

In [ ]:
sample_dspacing = powder.to_dspacing_with_calibration(sample, calibration=cal)

Vanadium

In [ ]:
vana = powder.load_vanadium(powgen.data.vanadium_file(), powgen.data.empty_instrument_file())

In [ ]:
vana_dspacing = powder.to_dspacing_with_calibration(vana, calibration=cal)

In [ ]:
vana_dspacing

In [ ]:
v = vana_dspacing.bins.concat('spectrum')
min_dspacing = sc.min(v.coords['dspacing']).value
max_dspacing = sc.max(v.coords['dspacing']).value
dspacing_edges = sc.linspace('dspacing',
                             min_dspacing,
                             max_dspacing,
                             200,
                             unit=v.coords['dspacing'].unit)
vana_histo = sc.histogram(v, bins=dspacing_edges)

In [ ]:
s = sample_dspacing.bins.concat('spectrum')
sample_corrected = s.bins / sc.lookup(vana_histo, dim='dspacing')
sample_corrected = sc.histogram(sample_corrected, bins=dspacing_edges)

In [ ]:
sample_corrected.plot()

In [ ]:
sample_dspacing.coords['two_theta'] = scn.two_theta(sample_dspacing)
vana_dspacing.coords['two_theta'] = scn.two_theta(vana_dspacing)

In [ ]:
two_theta = sc.linspace(dim='two_theta', unit='rad', start=0.0, stop=np.pi, num=16)
sample_focussed = sc.groupby(sample_dspacing, 'two_theta', bins=two_theta).bins.concat('spectrum')
vana_focussed = sc.groupby(sample_dspacing, 'two_theta', bins=two_theta).bins.concat('spectrum')

In [ ]:
norm = vana_histo = sc.histogram(vana_focussed, bins=dspacing_edges)
sample_focussed.bins /= sc.lookup(norm, dim='dspacing')
normalized = sc.histogram(sample_focussed, bins=dspacing_edges)

In [ ]:
normalized.plot(vmin=sc.scalar(0), vmax=sc.scalar(0.8))

In [ ]:
normalized

In [ ]:
angle = normalized.coords['two_theta'].values
angle = 0.5 * (angle[1:] + angle[:-1])
results = {
    f'{round(angle[group], 3)} rad': normalized['dspacing', :100]['two_theta', group]
    for group in range(2, 6)
}
sc.plot(results)